## Устанавливаем необходимые библиотеки

In [ ]:
pip install -q recbole ray kmeans_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 12.9 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import time

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity

import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

import torch
from recbole.model.general_recommender.multivae import MultiVAE

## Загружаем данные

Код с семинара:

In [ ]:
interactions_df = pd.read_csv('interactions_processed_kion.csv')
users_df = pd.read_csv('users_processed_kion.csv')
items_df = pd.read_csv('items_processed_kion.csv')

In [ ]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [ ]:
!mkdir recbox_data

In [ ]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

In [ ]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

In [ ]:
dataset = create_dataset(config)
logger.info(dataset)

In [ ]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

## Обучаем модели

In [ ]:
%%time
model_list = ["MultiVAE", "MultiDAE", "MacridVAE", "NeuMF", "RecVAE",
              "ItemKNN", "DMF", "ConvNCF", "LightGCN"]

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name,
                         dataset='recbox_data',
                         config_dict=parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)
    print("=="*20)

running MultiVAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-ffe82819-fe22-4eb8-992f-4401f79ca9dd.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|██████████████████| 13354/13354 [02:11<00:00, 101.74it/s, GPU RAM: 0.38 G/14.75 G]


It took 3.19 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0834), ('mrr@10', 0.1671), ('ndcg@10', 0.0816), ('hit@10', 0.3466), ('precision@10', 0.0462)])}
running MultiDAE...


command line args [-f /root/.local/share/jupyter/runtime/kernel-ffe82819-fe22-4eb8-992f-4401f79ca9dd.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [02:46<00:00, 80.21it/s, GPU RAM: 0.40 G/14.75 G]


It took 4.80 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0837), ('mrr@10', 0.1657), ('ndcg@10', 0.0814), ('hit@10', 0.3466), ('precision@10', 0.0463)])}
running MacridVAE...


...

It took 8.92 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0846), ('mrr@10', 0.1661), ('ndcg@10', 0.0818), ('hit@10', 0.3523), ('precision@10', 0.0469)])}
running ConvNCF...


command line args [-f /root/.local/share/jupyter/runtime/kernel-ffe82819-fe22-4eb8-992f-4401f79ca9dd.json] will not be used in RecBole
Evaluate   : 100%|███████████████████| 13354/13354 [09:34<00:00, 23.23it/s, GPU RAM: 1.59 G/14.75 G]


It took 22.12 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.061), ('mrr@10', 0.1361), ('ndcg@10', 0.0628), ('hit@10', 0.2648), ('precision@10', 0.0331)])}
running LightGCN...


command line args [-f /root/.local/share/jupyter/runtime/kernel-ffe82819-fe22-4eb8-992f-4401f79ca9dd.json] will not be used in RecBole
Evaluate   : 100%|███████████████████| 13354/13354 [06:00<00:00, 37.02it/s, GPU RAM: 1.59 G/14.75 G]


It took 15.61 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0792), ('mrr@10', 0.1685), ('ndcg@10', 0.0795), ('hit@10', 0.3385), ('precision@10', 0.0441)])}
CPU times: user 1h 31min, sys: 6min 14s, total: 1h 37min 15s
Wall time: 1h 44min 24s


## Лучшими моделями оказались `ItemKNN`, `RecVAE`, `MultiVAE`

In [ ]:
result = run_recbole(model="MultiVAE",
                     dataset="recbox_data",
                     config_dict=parameter_dict)

command line args [-f /root/.local/share/jupyter/runtime/kernel-38adbd9a-5a34-47f0-bcee-cd772e2c89d3.json] will not be used in RecBole
Max value of user's history interaction records has reached 20.9471766848816% of the total.
Evaluate   : 100%|███████████████████| 13354/13354 [03:40<00:00, 60.70it/s, GPU RAM: 0.67 G/14.75 G]


In [ ]:
model = MultiVAE(config, dataset)
checkpoint = torch.load("/content/saved/MultiVAE-Dec-13-2023_09-19-53.pth")
model.load_state_dict(checkpoint["state_dict"])

Max value of user's history interaction records has reached 23.254401942926535% of the total.


<All keys matched successfully>

Напишем функцию, чтобы сделать предсказания для каждого из пользователей

In [ ]:
model = model.to(config["device"])

In [ ]:
def recommend_to_user(user_id,
                      dataset,
                      model):
    if user_id in dataset.field2token_id[dataset.uid_field] and user_id != "[PAD]":
        model.eval()
        with torch.no_grad():
            uid_series = dataset.token2id(dataset.uid_field, [user_id])
            index = np.isin(dataset[dataset.uid_field].numpy(), uid_series)
            new_inter = dataset[index]
            new_inter = new_inter.to(config["device"])
            new_scores = model.full_sort_predict(new_inter)
            new_scores = new_scores.view(-1, test_data.dataset.item_num)
            new_scores[:, 0] = -np.inf
            recommended_item_indices = torch.topk(new_scores, 10).indices[0].tolist()
            recos = dataset.id2token(dataset.iid_field, [recommended_item_indices]).tolist()
        return recos
    return []

In [ ]:
recommendations = {}
users = dataset.field2token_id[dataset.uid_field]
for user_id in users:
    user_recs = recommend_to_user(user_id, dataset, model)
    if user_recs:
        recommendations |= {user_id: user_recs[0]}

In [ ]:
recs = {int(user_id): list(map(lambda x: int(x), recommendations[user_id])) for user_id in recommendations}

In [ ]:
with open("/content/MultiVAE_recs.pkl", "wb") as f:
    pickle.dump(recs, f)

## Добавив предсказания модели в сервис и протестировав их в боте, получил метрику `MAP@10 = 0.0881563` (>0.075)